<a href="https://colab.research.google.com/github/kovika98/reverse_pendulum/blob/dev/Reverse_Pendulum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>